In [1]:
import sys
sys.path.append("../")

from desci_sense.configs import default_init_parser_config
from desci_sense.shared_functions.dataloaders import scrape_post, convert_text_to_ref_post
from desci_sense.shared_functions.parsers.firebase_api_parser import FirebaseAPIParser, PromptCase

In [2]:
config = default_init_parser_config()

In [3]:
parser = FirebaseAPIParser(config=config)

2024-02-19 17:14:06.795 | INFO     | desci_sense.shared_functions.parsers.firebase_api_parser:set_md_extract_method:134 - Setting metadata extraction method to none...
2024-02-19 17:14:06.796 | INFO     | desci_sense.shared_functions.parsers.firebase_api_parser:__init__:97 - Loading parser model (type=mistralai/mistral-7b-instruct)...
/Users/ronentamari/anaconda3/envs/asensebot/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
2024-02-19 17:14:07.052 | INFO     | desci_sense.shared_functions.parsers.firebase_api_parser:set_kw_md_extract_method:138 - Setting keywords metadata extraction

In [4]:
parser.set_md_extract_method("citoid")

2024-02-19 17:14:08.023 | INFO     | desci_sense.shared_functions.parsers.firebase_api_parser:set_md_extract_method:134 - Setting metadata extraction method to citoid...


In [5]:
TEST_POST_TEXT_W_REF = """
I really liked this paper!
https://arxiv.org/abs/2402.04607
"""

In [6]:
post = convert_text_to_ref_post(TEST_POST_TEXT_W_REF)
combined = parser.process_ref_post_parallel(post)

In [13]:
combined["keywords"]["answer"]["valid_keywords"]

['self-citations', 'google-scholar', 'citation-fraud', 'scientific-evaluation']

In [7]:
post_proc = parser.post_process_result(**combined)

In [8]:
print(post_proc.model_dump()["semantics"])

@prefix ns1: <https://pcp-on-web.de/ontology/0.2/index-en.html#> .
@prefix ns2: <https://sense-nets.xyz/> .
@prefix ns3: <http://purl.org/spar/cito/> .

<http://purl.org/nanopub/temp/mynanopub#assertion> ns3:agreesWith <https://arxiv.org/abs/2402.04607> ;
    ns1:hasKeyword "citation-fraud",
        "google-scholar",
        "scientific-evaluation",
        "self-citations" ;
    ns2:readingStatus <https://arxiv.org/abs/2402.04607> .




In [9]:
presult = parser.process_text_parallel(TEST_POST_TEXT_W_REF)

In [10]:
print(presult.semantics.serialize())

@prefix ns1: <https://pcp-on-web.de/ontology/0.2/index-en.html#> .
@prefix ns2: <https://sense-nets.xyz/> .

<http://purl.org/nanopub/temp/mynanopub#assertion> ns1:hasKeyword "academic-integrity",
        "citation-fraud",
        "evaluation",
        "google-scholar",
        "preprint",
        "scientists" ;
    ns2:readingStatus <https://arxiv.org/abs/2402.04607> .




In [12]:
kwords = parser.extract_post_topics_w_metadata(post)

In [9]:
kwords["answer"]["valid_keywords"]

['SelfCitations',
 'CitationFraud',
 'CitationCartels',
 'GoogleScholar',
 'CitationManipulation',
 'ScientificEvaluation']

In [11]:
result = parser.process_ref_post(post)

In [4]:
chain = parser.prompt_case_dict[PromptCase.SINGLE_REF]["chain"]

In [5]:
chain

PromptTemplate(input_variables=['input'], template='{input}')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x28f0f5410>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x28fcbb850>, model_name='mistralai/mistral-7b-instruct', temperature=0.6, openai_api_key='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', openai_api_base='https://openrouter.ai/api/v1', openai_proxy='')
| TagTypeParser(allowed_tags=['endorses', 'disagrees', 'agrees', 'watching', 'reading', 'listening', 'default', 'review', 'recommendation', 'question', 'quote', 'discussion', 'event', 'job', 'announce'])

In [6]:
kw_chain = parser.kw_extraction["chain"]
kw_chain

PromptTemplate(input_variables=['kw_input'], template='{kw_input}')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x28fcc3850>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x28fcd2cd0>, model_name='mistralai/mistral-7b-instruct', temperature=0.6, openai_api_key='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', openai_api_base='https://openrouter.ai/api/v1', openai_proxy='')
| KeywordParser(max_keywords=6)

In [8]:
post = convert_text_to_ref_post(TEST_POST_TEXT_W_REF)

In [9]:
result = parser.process_ref_post_st(post)

In [12]:
parser_prompt = result["full_prompt"]

In [10]:
kw_result = parser.extract_post_topics_w_metadata(post)

In [13]:
kw_prompt = kw_result["full_prompt"]

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

In [16]:
map_chain = RunnableParallel(semantics=chain, keywords=kw_chain)

In [17]:
full_res = map_chain.invoke({"kw_input": kw_prompt, "input": parser_prompt})

In [19]:
full_res["keywords"]

{'reasoning': '[Reasoning Steps]\n\n[System error: failed to extract reasoning steps since the generated output was in an invalid format]\n\n[Candidate Tags]\n\n[System error: failed to extract candidate tags since the generated output was in an invalid format.]',
 'final_answer': '1. #citation-fraud\n2. #Google-Scholar\n3. #scientific-evaluation\n4. #inflation\n5. #manipulability\n6. #citation-cartels',
 'valid_keywords': ['citation-cartels',
  'manipulability',
  'scientific-evaluation',
  'Google-Scholar',
  'citation-fraud',
  'inflation']}

In [27]:
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)



In [26]:
%%timeit

map_chain.invoke({"topic": "bear"})

1.21 s ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
joke_chain.invoke({"topic": "bear"})

1.3 s ± 240 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit

poem_chain.invoke({"topic": "bear"})

1.3 s ± 302 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
